In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
ROOT='/content/drive/My Drive/dl/推荐评论/'
BERT_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/新建文件夹/bert-base-chinese/'
BERT_TOK_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/bert-base-chinese-vocab.txt'
MAX_LEN=22

In [0]:
!pip install pytorch_pretrained_bert
!pip install transformers
!pip install scikit-learn==0.21.3

In [0]:
import sklearn
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from pytorch_pretrained_bert import BertModel, BertTokenizer
root='/content/drive/My Drive/google-quest-challenge/'
import numpy as np
import time
import tqdm

import math
import scipy
import torch.nn.functional as F
import random
import transformers, sys, os, gc
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)
from torch.utils.data import DataLoader, Dataset,RandomSampler, SequentialSampler
from transformers.modeling_bert import BertPreTrainedModel 
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from tqdm import tqdm_notebook
from sklearn.metrics import roc_curve 
import numpy as np

In [0]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
setup_seed(42)

In [0]:
labels,train_data=[],[]
test=[]
weight=[]
lenth = []

In [0]:
with open (ROOT+'data/train.txt', 'r') as f:
  for line in f:
    label,text = line.split('\t')
    labels.append(int(label))
    train_data.append(text.replace('\n',''))

In [0]:
for label in labels:
  if label == 1:
    weight.append(3)
  else:
    weight.append(1)


In [0]:
tokenizer=BertTokenizer.from_pretrained(BERT_TOK_ROOT)
train_idx = []
train_mask = []
train_segment = []

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [0]:
for line in range(len(train_data)):
  token = tokenizer.tokenize(train_data[line])
  token = ["[CLS]"] + token + ["[SEP]"]
  lenth.append(len(token))
  if len(token)> MAX_LEN:
      raise IndexError("Token length more than max seq length!")

  mask = [1]*len(token)+ [0]*(MAX_LEN-len(token))
  sent = [0]*MAX_LEN
  idx = tokenizer.convert_tokens_to_ids(token)
  idx = idx + (MAX_LEN-len(idx))*[0]
  train_idx.append(idx)
  train_mask.append(mask)
  train_segment.append(sent)

# **统计特征**

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [0]:
text_encoder = Pipeline([
    ('Text-TF-IDF', TfidfVectorizer(ngram_range=(1, 3))),
    ('Text-SVD', TruncatedSVD(n_components = 600))], verbose=True)

In [0]:
text_encoder = torch.load('/content/drive/My Drive/dl/推荐评论/text_encoder.pkl')

In [0]:
w20_train_tfidf = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_tfidf.pkl')

In [0]:
tfidf=text_encoder.transform(train_data+w20_train_tfidf)

In [0]:
w20_lenth = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_lenth.pkl').tolist()

In [0]:
len(w20_lenth)

56366

In [0]:
true_tfidf=tfidf[:16000]
w_tfidf=tfidf[16000:]

In [0]:
lenths = np.array(lenth)
lenths=lenths.reshape(-1,1)
w_lenth = np.array(w20_lenth).reshape(-1,1)


In [0]:
w_tfidf.shape

(56366, 900)

In [0]:
use = np.hstack( (lenths,true_tfidf))
w_use = np.hstack((w_lenth,w_tfidf))
print(use.shape)


(16000, 901)


# **Dataset**

In [0]:
class EvalDataset(Dataset):
  def __init__(self,train_idx,train_mask,train_segment,use,label=None):
    self.train_idx = torch.tensor(train_idx,dtype=torch.int).long()
    self.train_mask = torch.tensor(train_mask,dtype=torch.int).long()
    self.train_segment = torch.tensor(train_segment,dtype=torch.int).long()
    self.use = torch.tensor(use , dtype=torch.float32)
    self.label = None
    if label is not None:
      self.label = torch.tensor(label,dtype=torch.float32)
  def __len__(self):
    return len(self.train_idx)
  def __getitem__(self,item):
    if self.label is not None:

      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'train_segment' :self.train_segment[item],
        'use':self.use[item],
        'label' : self.label[item]
    }
    else:
      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'use' : self.use[item],
        'train_segment' :self.train_segment[item],
      }
    return {key: value for key, value in output.items()}

In [0]:
def get_ids(x,y):
  z=[]
  for i in y:
    z.append(x[i])
  return z

In [0]:
class GELU(nn.Module):
    """
    Paper Section 3.4, last paragraph notice that BERT used the GELU instead of RELU
    """

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x
class SoftmaxPooling(nn.Module):
    def __init__(self, dim=1):
        super(self.__class__, self).__init__()
        self.dim = dim
        
    def forward(self, x):
        return (x * x.softmax(dim=self.dim)).sum(dim=self.dim)  
class Attention1(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention1, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / torch.sum(a, 1, keepdim=True) + 1e-10
        
        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1) 
class SoftmaxPooling(nn.Module):
    def __init__(self, dim=1):
        super(self.__class__, self).__init__()
        self.dim = dim
        
    def forward(self, x):
        return (x * x.softmax(dim=self.dim)).sum(dim=self.dim)   
class lstm_att(nn.Module):
    def __init__(self,max_features,embed_size,hid_size):
        super(lstm_att, self).__init__()

        self.embedding=nn.Embedding(max_features,embed_size)
        self.embedding_dropout = SpatialDropout(0.1)

        ebsize=hid_size
        self.att11=Attention1(ebsize*2,22)
        self.att22=Attention1(ebsize*2,22)
        self.pool1=SoftmaxPooling()
        self.drop=nn.Dropout(0.3)
        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])
        self.lstm1 = nn.LSTM(embed_size, ebsize, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(ebsize*2, ebsize, bidirectional=True, batch_first=True)
        self.linear11=nn.Linear(ebsize*10,ebsize*10)
        self.linear12=nn.Linear(ebsize*10,ebsize*10)
        self.droplist = nn.ModuleList([nn.Dropout(0.7) for _ in range(8)])
        self.linear_out11 = nn.Linear(ebsize*10, 1)
        self.linear = nn.Linear(ebsize*8,ebsize*8)
    def forward(self,train_idx):
        h_embedding = (self.embedding(train_idx))
        
        h_lstm1, _ = self.lstm1((h_embedding))
        h_lstm2, _ = self.lstm2(self.drop((h_lstm1)))
        att1=self.att11(h_lstm1)
        att2=self.att22(h_lstm2)
        pool1 = self.pool1(h_embedding+h_lstm1+h_lstm2)
        pool2 = torch.mean(h_embedding+h_lstm1+h_lstm2, 1)
        # global max pooling
        pool3, _ = torch.max(h_embedding+h_lstm1+h_lstm2, 1)
        h_conc = torch.cat((att1,att2,pool1,pool2,pool3), 1)
  
        h_conc_linear1 = F.relu(self.linear11((self.dropouts[0](h_conc))))
        h_conc_linear2 = F.relu(self.linear12((self.dropouts[1](h_conc))))

        result1 = self.linear_out11((h_conc+h_conc_linear1+h_conc_linear2))
        return (result1,)

In [0]:
bert_model_config = BERT_ROOT+'/bert_config.json'
bert_config = BertConfig.from_json_file(bert_model_config)
bert_config.num_labels = 1
bert_config.output_hidden_states=True


In [0]:
def my_loss(input,target,weight=None):
  input=torch.sigmoid(input)
  if weight is not None:
    loss =torch._C._nn.binary_cross_entropy(input,target)
    loss*=weight
    return loss.sum()
  else:
    loss = torch._C._nn.binary_cross_entropy(input,target)
    return loss.sum()

In [0]:
def train_model(train_loader ,train_shape, optimizer ,scheduler, criterion,criterion2, config=None , key = True):
  model.train()
  acc = 0
  auc = 0
  avg_loss = 0
  optimizer.zero_grad()
  for idx, batch in tqdm_notebook(enumerate(train_loader),mininterval=2,desc='--Training',leave=False):
    input_ids = batch['train_idx'].to(device)
    input_mask = batch['train_mask'].to(device)
    input_segments = batch['train_segment'].to(device)
    label = batch['label'].to(device).view(-1)
    use = batch['use'].to(device)

    output_train = model(input_ids.long()     )
    #eps=0.1
    #one_hot=(1-eps)*label+(1-label)*eps/(2-1)

    loss = criterion(output_train[0].view(-1),label) 
    #loss = my_loss(output_train.view(-1),label,weight)
    avg_loss += loss.item()
    loss.backward()
    optimizer.step()
    scheduler.step()
    acc += ((torch.sigmoid(output_train[0].cpu().view(-1))>0.5).int()==label.cpu()).sum()
    #auc += sklearn.metrics.roc_auc_score( (label.cpu().int()),torch.sigmoid(output_train.view(-1)).detach().cpu().numpy())
    fgm.attack()
    output_train = model( input_ids.long())
    loss_adv = criterion(output_train[0].view(-1),label)     
    loss_adv.backward()
    fgm.restore()

    optimizer.step()
    scheduler.step()
    
    
    ema.update()
    optimizer.zero_grad()
    
  return avg_loss/train_shape,acc.item()/train_shape,auc/len(train_loader)

In [0]:
def valid_model(valid_loader ,val_shape,  criterion, batch_size ,config=None):
  model.eval()
  acc = 0
  auc = 0
  avg_loss = 0
  pred = np.zeros((val_shape))
  ema.apply_shadow()
  with torch.no_grad():
    for idx, batch in tqdm_notebook(enumerate(valid_loader),mininterval=2,desc='--Valing',leave=False):
      input_ids = batch['train_idx'].to(device)
      input_mask = batch['train_mask'].to(device)
      input_segments = batch['train_segment'].to(device)
      label = batch['label'].to(device).view(-1)
      use = batch['use'].to(device)
      output_train = model(input_ids.long())[0]
             
      loss = criterion(output_train.view(-1),label)
      #loss = my_loss(output_train.view(-1),label,weight)
      pred[idx*batch_size : (idx+1)*batch_size] = output_train.view(-1).detach().cpu().squeeze().numpy()
      avg_loss += loss.item()
      acc += ((torch.sigmoid(output_train.cpu().view(-1))>0.5).int()==label.cpu()).sum()
      if input_ids.size()[0]!=1:

        auc += sklearn.metrics.roc_auc_score( (label.cpu().int()),torch.sigmoid(output_train.view(-1)).detach().cpu().numpy())
  ema.restore()
  return avg_loss/val_shape,acc.item()/val_shape,auc/len(valid_loader),torch.sigmoid(torch.tensor(pred)).numpy()

In [0]:
class EMA():
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

In [0]:
class FGM():
  def __init__(self,model):
    self.model=model
    self.backup={}
  def attack(self,epsilon=0.5,emb_name='emb'):
    for name,param in self.model.named_parameters():
      if param.requires_grad and emb_name in name:
        self.backup[name]=param.data.clone()
        if (param.grad)== None:
          continue
        norm=torch.norm(param.grad)
        if norm!= 0 and not torch.isnan(norm):
          r_at=epsilon * param.grad/norm
          param.data.add_(r_at)
  def restore(self,emb_name='emb'):
    for name,param in self.model.named_parameters():
      if param.requires_grad and emb_name in name:
        assert name in self.backup
        param.data=self.backup[name]
    self.backup={}

In [0]:
BATCH_SIZE =70
NUM_EPOCH=30
device = torch.device('cuda')

In [0]:
floder = StratifiedKFold(n_splits=5,random_state=0,shuffle=False)
predict = np.zeros(len(train_idx))


In [0]:
 
 
def transfer_model(pretrained_file, model):
    '''
    只导入pretrained_file部分模型参数
    tensor([-0.7119,  0.0688, -1.7247, -1.7182, -1.2161, -0.7323, -2.1065, -0.5433,-1.5893, -0.5562]
    update:
        D.update([E, ]**F) -> None.  Update D from dict/iterable E and F.
        If E is present and has a .keys() method, then does:  for k in E: D[k] = E[k]
        If E is present and lacks a .keys() method, then does:  for k, v in E: D[k] = v
        In either case, this is followed by: for k in F:  D[k] = F[k]
    :param pretrained_file:
    :param model:
    :return:
    '''
    pretrained_dict = torch.load(pretrained_file)  # get pretrained dict
    model_dict = model.state_dict()  # get model dict
    # 在合并前(update),需要去除pretrained_dict一些不需要的参数
    pretrained_dict = transfer_state_dict(pretrained_dict, model_dict)
    model_dict.update(pretrained_dict)  # 更新(合并)模型的参数
    model.load_state_dict(model_dict)
    return model
 
 
def transfer_state_dict(pretrained_dict, model_dict):
    '''
    根据model_dict,去除pretrained_dict一些不需要的参数,以便迁移到新的网络
    url: https://blog.csdn.net/qq_34914551/article/details/87871134
    :param pretrained_dict:
    :param model_dict:
    :return:
    '''
    # state_dict2 = {k: v for k, v in save_model.items() if k in model_dict.keys()}
    state_dict = {}
    for k, v in pretrained_dict.items():
        if k in model_dict.keys():
            # state_dict.setdefault(k, v)
            state_dict[k] = v
        else:
            print("Missing key(s) in state_dict :{}".format(k))
    return state_dict
 
 


In [0]:
wlabel_label=torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_label_bin.pkl')
wlabel_train_idx = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_idx.pkl')
wlabel_train_mask = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_mask.pkl')
wlabel_train_segment = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_segment.pkl')
#wlabel_train_use=torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_tfidf.pkl')

In [0]:
use_shape = use.shape[1]
use = (use).tolist()

In [0]:
for fold,(train, test) in enumerate(floder.split(train_idx,labels)):
  ags=3000
  t_idx = get_ids(train_idx , train)+wlabel_train_idx
  t_mask = get_ids(train_mask , train)+wlabel_train_mask
  t_segment = get_ids(train_segment , train)+wlabel_train_segment
  t_weight = get_ids(weight , train)
  #t_label = get_ids(labels , train)# (np.hstack((np.array(get_ids(labels , train)),wlabel_label)))
  t_label = (np.hstack((np.array(get_ids(labels , train)),wlabel_label)))
  t_use = (np.vstack((np.array(get_ids(use , train)),w_use)))
  #t_use=(np.array(get_ids(use , train)))
  print(t_use.shape)
  valid_idx = get_ids(train_idx , test)
  valid_mask = get_ids(train_mask , test)
  valid_segment = get_ids(train_segment , test)
  valid_weight = get_ids(weight , test)
  valid_label = get_ids(labels , test)
  valid_use = get_ids(use , test)
  train_shape = len(t_idx)
  valid_shape = len(valid_idx)
  train_set = EvalDataset(t_idx , t_mask , t_segment , t_use ,t_label)
  vla_set = EvalDataset(valid_idx , valid_mask , valid_segment ,valid_use ,valid_label)
  train_loader = DataLoader (train_set , batch_size=BATCH_SIZE,shuffle=True)
  valid_loader = DataLoader (vla_set , batch_size=BATCH_SIZE,shuffle=False)
  
  model = lstm_att(bert_config.vocab_size,256,128)
  #model = transfer_model('/content/drive/My Drive/dl/美团/LSTM/best_pre_acc_1.pt',model) #使用预训练模型
  model.zero_grad();
  model.to(device);
  fgm = FGM(model)
  ema=EMA(model,0.995)
  ema.register()
  def is_backbone(n):
        return "bert" in n



  

  optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, eps=4e-5,weight_decay=0.8)
  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.05, num_training_steps= 30*len(train_loader))
  criterion = nn.BCEWithLogitsLoss()
  criterion2 = nn.CrossEntropyLoss()
  best_avg_loss = -100
  best_score = -1
  best_param_score = None
  key = True
  for epoch in range(NUM_EPOCH):
    if epoch>2:
      key = False
    torch.cuda.empty_cache()
    start_time = time.time()
    train_loss,train_acc,train_auc=train_model(train_loader ,train_shape, optimizer ,scheduler, criterion,criterion2, config=None ,key = key)
    val_loss,val_acc,val_auc,pred=valid_model(valid_loader ,valid_shape,  criterion, BATCH_SIZE ,config=None)
    elapsed_time = time.time() - start_time
    print('epoch{}/{} , train_loss: {} ,train_acc:{} ,train_auc: {} \n \n val_loss :{} , val_acc :{} , val_auc:{} ,time: {}\n'.format(epoch+1,NUM_EPOCH,train_loss,train_acc,train_auc,val_loss,val_acc,val_auc,elapsed_time))
    if best_score < val_auc:
      best_score = val_auc
      best_param_score = model.state_dict()
      predict[test] = pred
      torch.save(best_param_score,ROOT+'LSTM/best_param_score_{}'.format(fold+1))
      torch.save(ema,ROOT+'LSTM/EMA_{}'.format(fold+1))
  del model, optimizer, criterion, scheduler
  torch.cuda.empty_cache()
  del valid_loader, train_loader, vla_set, train_set

/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be com

epoch26/30 , train_loss: 0.0020397168243798816 ,train_acc:0.9409247318046439 ,train_auc: 0.0 
 
 val_loss :0.003539858306758106 , val_acc :0.9096875 , val_auc:0.9556410748530304 ,time: 25.41771101951599



/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be com

/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be com

epoch27/30 , train_loss: 0.0020994051474773775 ,train_acc:0.9403897868895121 ,train_auc: 0.0 
 
 val_loss :0.003528400775976479 , val_acc :0.905 , val_auc:0.9559860845966471 ,time: 25.417526960372925



...

epoch29/30 , train_loss: 0.0021291024039205536 ,train_acc:0.9383665620888574 ,train_auc: 0.0 
 
 val_loss :0.003281424784518734 , val_acc :0.9118474523288528 , val_auc:0.9621326042670594 ,time: 25.86462664604187



/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be com

/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be com

epoch30/30 , train_loss: 0.002145591526239038 ,train_acc:0.9374701808665983 ,train_auc: 0.0 
 
 val_loss :0.003319644430030991 , val_acc :0.9109096592685214 , val_auc:0.961738326863205 ,time: 25.64148187637329



/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1266: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be com

In [0]:
sklearn.metrics.roc_auc_score(labels,predict)

0.96193177831517